<center>

<h1 style="text-align:center"> Pattern Matching </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Review


Previously:

* Tuples, Records, Variants
* Polymorphism
* Lists, Option

This lecture:

* Pattern Matching

## Pattern Matching

* Pattern matching is data deconstruction
  + Match on the *shape* of data
  + Extract part(s) of data
  
### Syntax

```ocaml
match e with
| p1 -> e1
| p2 -> e2
...
| pn -> en
```

* p1 ... pn are patterns.

## Pattern Matching on Lists

```
type 'a list = [] | :: of 'a * 'a list
```

* For lists, the patterns allowed follow from the constructors
  + The pattern `[]` matches the value `[]`.
  + The patterh `h::t`
    - matches `2::[]`, binding `h` to `2` and `t` to `[]`.
    - matches `2::3::[]`, binding `h` to `2` and `t` to `3::[]`.
  + The pattern `_` is a **wildcard pattern** and matches anything. 

In [ ]:
let list_status l =
  match l with
  | [] -> print_endline "The list is empty"
  | h::t -> Printf.printf "The list is non-empty. Head = %d\n%!" h

In [ ]:
list_status []

In [ ]:
list_status [1;2;3]

In [ ]:
list_status (2::[3;4])

## Why pattern matching is THE GREATEST

1. You cannot forget to match a case (Exhaustivity warning)

In [ ]:
let list_status l = 
  match l with 
  | [] -> print_endline "The list is empty"
  | h1::h2::t -> Printf.printf "The list is non-empty. 2nd element = %d\n%!" h2

## Why pattern matching is THE GREATEST

1. You cannot forget to match a case (Exhaustivity warning)
2. You cannot duplicate a case (Unused case warning)

In [ ]:
let list_status l = 
  match l with 
  | [] -> print_endline "The list is empty"
  | h::t -> Printf.printf "The list is non-empty. Head = %d\n%!" h
  | h1::h2::t -> Printf.printf "The list is non-empty. 2nd element = %d\n%!" h2

## Why pattern matching is THE GREATEST

1. You cannot forget to match a case (Exhaustivity warning)
2. You cannot duplicate a case (Unused case warning)

<h2> Pattern matching leads to elegant, concise, beautiful code <h2>

## Length of list

In [ ]:
let rec length l =
  match l with
  | [] -> 0
  | h::t -> 1 + length t

What is wrong with this code?

## Length of list (tail recursive)

In [ ]:
let rec length' l acc =
  match l with
  | [] -> acc
  | h::t -> length' t (1+acc)
  
let length l = length' l 0

In [ ]:
length [1;2;3;4]

## Exercise

Implement the reverse of a list.

In [104]:
let rev_list l = failwith "not implemented"

error: compile_error

In [105]:
assert (rev_list [1;2;3] = [3;2;1])

error: runtime_error

## Exercise

Implement the append of two lists.

In [109]:
[1;2;3] @ [4;5;6]

- : int list = [1; 2; 3; 4; 5; 6]


In [106]:
let append l1 l2 = failwith "not implemented"

val append : 'a -> 'b -> 'c = <fun>


In [107]:
assert (append [1;2;3] [4;5;6] = [1;2;3;4;5;6])

error: runtime_error

## Match ordering

The patterns are matched in the order that they are written down.

In [ ]:
let is_empty l =
  match l with
  | [] -> true
  | _ -> false

## Nested Matching



In [ ]:
type color = Red | Green | Blue

type point = {x : int; y : int}

type shape = 
  | Circle of point * float (* center, radius *)
  | Rect of point * point   (* lower-left, upper-right *)
  | ColorPoint of point * color

## Nested Matching

Is the first shape in a list of shapes a red point?

In [ ]:
let is_hd_red_circle l = 
  match l with
  | ColorPoint(_,Red)::_ -> true
  | _ -> false

## Nested Matching

Print the coordinates if the point is green.

In [ ]:
let rec print_green_point l =
  match l with
  | [] -> ()
  | ColorPoint({x;y}, Green)::tl -> 
      Printf.printf "x = %d y = %d\n%!" x y;
      print_green_point tl
  | _::tl -> print_green_point tl

In [ ]:
print_green_point [Rect ({x=1;y=1},{x=2;y=2});
                   ColorPoint ({x=0;y=0}, Green);
                   Circle ({x=1;y=3}, 5.4);
                   ColorPoint ({x=4;y=6}, Green)]

## When do you use ";"

When you evaluate an expression just the effect, you can sequence the expression with a semi-colon.

```ocaml
let () = print_endline "Hello, world!" in
e
```

is equivalent to:

```ocaml
print_endline "Hello, world!";
e
```

Latter is considered better style.

## Exceptions

* OCaml has support for exceptions. 
  + Similar to the ones found in C++ & Java.
* Exceptions are (mostly) just variants.

```ocaml
type exn
exception MyException of string
```

* The type `exn` is an **extensible variant**.
  + New constructors of this type can be added after its original declaration.
* Exceptions are raised with `raise e` where `e` is of type `exn`.
* Handling exceptions is similar to pattern matching.

## Find the green point

Given a list of shapes return a point whose colour is green. Otherwise, raise `NoGreenPoint` exception.

In [ ]:
exception NoGreenPoint

let rec find_green_point l = 
  match l with
  | [] -> raise NoGreenPoint
  | h::tl ->
    match h with
    | ColorPoint (_, Green) -> h
    | _ -> find_green_point tl

## Find the green point

In [ ]:
find_green_point []

In [ ]:
find_green_point [Rect ({x=1;y=1},{x=2;y=2}); ColorPoint ({x=0;y=0}, Green)]

## Handling the exception

Given a list of shapes return `Some p` where `p` is a green point. Otherwise, return `None`.

In [ ]:
let find_green_point_opt l =
  try Some (find_green_point l) with
  | NoGreenPoint -> None

In [ ]:
find_green_point_opt []

In [ ]:
find_green_point_opt [Rect ({x=1;y=1},{x=2;y=2}); ColorPoint ({x=0;y=0}, Green)]

## Exceptions: Recommendations

* Avoid exceptions in your code.
  + Unhandled exceptions are runtime errors; aim to avoid this.
* No exhaustiveness check for exceptions (why?).
* Whenever you might need to use exceptions, think whether you can replace that with

```ocaml
type 'a option = None | Some of 'a
```

or 

```ocaml
type ('a,'b) result = Ok of 'a | Error of 'b
```

<center>

<h1 style="text-align:center"> Fin. </h1>
</center>